In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def write_to_file(data, filename):
    with open(filename, 'w') as f:
        for line in data:
            f.write(line + '\n')

def read_from_file(filename):
    with open(filename, 'r') as f:
        return np.array(f.read().splitlines())

# Dataframe Train

In [3]:
train = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\train.csv").fillna('')
test = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test.csv").fillna('')
submission = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\sample_submission.csv")

In [4]:
y_train_origin = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test_labels.csv")
classes = y_train_origin.columns.values[1:]
classes[0] 

'toxic'

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True, smooth_idf=True, sublinear_tf=True)

In [6]:
X_train_origin = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_train_cleaned_vanilla.txt")

In [7]:
X_test_origin = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_test_cleaned_vanilla.txt")

In [8]:
train.loc[:, 'comment_text'] = X_train_origin
train = train[:1000]


In [9]:
from sklearn.model_selection import train_test_split
# from nltk import ngrams, bigrams, trigrams
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV


# Model

In [34]:
from sklearn.metrics import classification_report, confusion_matrix


def trainGrid (train, num_col, label, vectorizer = vec):
    X_train, X_test, y_train, y_test = train_test_split(train, train[label], test_size=0.3, random_state= 50)
    
    ########### transform
    X_train_trans = vectorizer.fit_transform(X_train['comment_text'])
    X_test_trans = vectorizer.transform(X_test['comment_text'])

    def pr(y_i, y):
        p = X_train_trans[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)
    def get_mdl(y):
        y = y.values
        r = np.log(pr(1,y) / pr(0,y))
        # print(r)
        return y,r
    
    y,r = get_mdl(X_train['toxic'])
    x_nb = X_train_trans.multiply(r)
    
    param_grid = {'C':[1,10,100,1000], 'loss':['hinge', 'square_hinge']}
    grid = GridSearchCV(LinearSVC(),param_grid,refit = True, cv= 3, verbose=2)
    grid.fit(x_nb, y)
    print(X_test_trans.shape)
    prediction = grid.predict(X_test_trans.multiply(r))
    print(classification_report(y_test,prediction))
    print(confusion_matrix(y_test, prediction)) 

    model = CalibratedClassifierCV(LinearSVC('l2', loss = grid.best_params_['loss'], cv = 3, C = grid.best_params_["C"]))
    fitted_model = model.fit(vec.fit_transform(train))
    

trainGrid(train, 0, 'toxic', vec)

    

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END .............................C=1, loss=square_hinge; total time=   0.0s
[CV] END .............................C=1, loss=square_hinge; total time=   0.0s
[CV] END .............................C=1, loss=square_hinge; total time=   0.0s
[CV] END ...................................C=10, loss=hinge; total time=   0.0s
[CV] END ...................................C=10, loss=hinge; total time=   0.0s
[CV] END ...................................C=10, loss=hinge; total time=   0.0s
[CV] END ............................C=10, loss=square_hinge; total time=   0.0s
[CV] END ............................C=10, loss=square_hinge; total time=   0.0s
[CV] END ............................C=10, loss=s

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The 

[CV] END ..................................C=100, loss=hinge; total time=   0.0s
[CV] END ..................................C=100, loss=hinge; total time=   0.0s
[CV] END ...........................C=100, loss=square_hinge; total time=   0.0s
[CV] END ...........................C=100, loss=square_hinge; total time=   0.0s
[CV] END ...........................C=100, loss=square_hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END ..........................C=1000, loss=square_hinge; total time=   0.0s
[CV] END ..........................C=1000, loss=square_hinge; total time=   0.0s
[CV] END ..........................C=1000, loss=square_hinge; total time=   0.0s
(300, 3620)
              precision    recall  f1-score   support

           0       0.95      0.99      0.9

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
